**Imports**

In [34]:
#Project by:
#Zachary Gerstenfeld....._______________

#enter the path to get to CDs_and_Vinyl folder here
PATH_TO_CDS = '../devided_dataset_v2/CDs_and_Vinyl/'

import json
import pandas as pd
from sklearn.naive_bayes import GaussianNB
from sklearn.model_selection import train_test_split
import numpy as np

from sklearn.feature_extraction import DictVectorizer
v = DictVectorizer(sparse=False, sort=True)

from sklearn.feature_extraction.text import TfidfVectorizer
tfidf = TfidfVectorizer()

# use sklearn's tfidf or just count

**Pre Processing**

In [17]:
x = pd.read_json(PATH_TO_CDS + 'train/review_training.json')
y = pd.read_json(PATH_TO_CDS + 'train/product_training.json')

z = pd.merge(x,y,'inner','asin')
zgrouped = z.groupby("asin")

In [52]:
tests = range(5)
unique_items = z["asin"].nunique()
# print(unique_items)

asin = z["asin"].values
avg_votes = np.zeros(len(asin))

for i in tests: # eventually all
    item = zgrouped.get_group(asin[i])
    avg_votes[i] = item["vote"].fillna(0).astype(int).mean()

print(avg_votes[0:5])
    


[6.78767922 6.78767922 6.78767922 6.78767922 6.78767922]


In [5]:


# pandas function - try to join by asin
# then take awesomeness col and =y


# z.columns
z=z.dropna(subset="summary")
x = z["summary"]
y = z["awesomeness"]
y=y[:20000]
x=x[:20000]

#!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!IMPORTANT!!!!!!!!!!!!!!!!!!!!
#note: many reviews/item, so need to combine whatever numerical results from reviews -> 1 value, then use that to predict awesomeness for 1 row of y. 
# Y can't repeat if we're going for practical results!!! 

# x = v.fit_transform(x)
# print(type(x))
# y = v.fit_transform(y)

######################################
#if i could just get x, y to be numpy arrays, I can reshape them if necessary, so it'll probs work


x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.5, random_state=2)

# make vocab = fit, transform -> numbers = transform
x_train = tfidf.fit_transform(x_train).toarray()
x_test = tfidf.transform(x_test).toarray()

gnb = GaussianNB()
y_pred = gnb.fit(x_train, y_train).predict(x_test)
print(f"Number of mislabeled points out of a total {x_test.shape[0]} points : {(y_test != y_pred).sum()}")

<bound method NDFrame.head of                                     asin                        reviewerID  \
0       6673F1740E03573BCD64238FE711FC69  9C856D4A18E1355783B3B28B7ECC1848   
1       6673F1740E03573BCD64238FE711FC69  607256A8DAC2F8AC4A7DD9FBE88362AC   
2       6673F1740E03573BCD64238FE711FC69  023DC0A6B4724F8E17C659F086DC99AB   
3       6673F1740E03573BCD64238FE711FC69  1F10DD5EF71B7A52F102C90596F17232   
4       6673F1740E03573BCD64238FE711FC69  9C856D4A18E1355783B3B28B7ECC1848   
...                                  ...                               ...   
770781  928CCBB9A92106966827DE24214976BC  09DF81587BE9477D564DBD3E23106792   
770782  431D15AA55CF51F5BF1E7D5E4B854B22  7E27095C8C499EAFA81C96F0B68210AB   
770783  2DA5B27AFDEF44C647111CAC58E92868  0326A4CBC5EC6125B60EB77D78C319E6   
770784  0E27343B95E949FC7EB4F03E0AD873A6  72E794FAD2B0A5BD070F4E843F1EE4F4   
770785  99451B018E70D8F75DE529F960E36858  082AB116F37B1944448CEE5D9FB61CA1   

        unixReviewTime  vote  ver

TypeError: Index(...) must be called with a collection of some kind, 'summary' was passed